In [1]:
import os
import numpy as np
import pandas as pd
from scipy.linalg import expm
from scipy.optimize import fsolve

# Isotope Ratio Method

This notebook demonstrates the functions for inferring the produced plutonium from an isotope ratio measurement.
The output of a reactor simulation has been processed and is provided to illustrate the method.

## Processed Output from the Reactor Simulation
- Time-averaged neutron spectrum $\phi_E$
- Plutonium density vector in the simulated fuel cell 
- Time steps of the reactor operation


A core approximation in this simplified IRM implementation is that a reactor
is operated for multiple cycles that are very similar and can thus be 
approximated with an average cycle, which is called an 'average batch' in the
paper. It is assumed that after one such cycle, the entire core is emptied and
the reactor is refueled with fresh fuel.

## Read Input Files

Plutonium.csv contains isotopic vector of plutonium in the fuel, extracted from one `SERPENT` simulation.
The nuclides Np239 and U239 are included, since the simulation did not use a cooling time after the final burnup step.
Spectra.csv contains the neutron spectrum for each step of the simulation (corresponding to the steps in the plutonium file), measured in the target location for the irm sample.

In [2]:
spectra = pd.read_csv('spectra.csv', index_col=0)
plutonium = pd.read_csv('plutonium.csv', index_col=0)

For the IRM calculation presented below, we need to derive several parameters from the simulation output:
- $Pu_0$ : the plutonium density in the fuel at the end of the cycle when the fuel is extracted

In [3]:
day_0 = plutonium.index[-1]
t_0 = day_0 * 60 * 60 * 24 #convert from days to seconds
pu_0 = plutonium.loc[day_0,:].sum()

- $\overline{\phi_E}$ : the average neutron spectrum at the sample location

In [4]:
phi_E_mean = spectra.mean(axis=1)

- $\Phi_0$ : the total neutron fluence received by the sample location.
The neutron fluence is approximated as $\int dE dt \phi_E\left(t\right)\approx \sum_E\overline{\phi_E} \times t_0$

In [5]:
flux_mean = phi_E_mean.sum()
phi_0 = flux_mean * t_0

## Evolution of the Isotopic Vector of Indicator Elements

The differential equations that describe the changes of the isotopic concentration $\vec{N}$ can be expressed with help of the transition matrix $\textbf{A}$:
$$\frac{d\vec{N}}{dt} = \textbf{A}\cdot\vec{N}.$$
$\vec{N}$ is the vector of number densities of the nuclides and $\textbf{A}$ contains the probability for each reaction that produces or destroys a nuclide.
In the cells below, we compute the components of the matrix $\textbf{A}$ and call them reaction rates, which is a microscopic reaction rate (per target particle).

#### Microscopic Reaction Rate

We use preprocessed cross-section data from the [JANIS 4.0 database](https://www.oecd-nea.org/jcms/pl_39910/janis).
The cross-section librarys are provided on a specific energy grid and we tally the neutron spectrum on the same grid, to facilitate calculating the reaction rate or the one-group cross-section.

In [6]:
xs_janis = pd.read_csv('janis_xs_B_Ti.csv', index_col=0, header=[0,1])
xs_values = xs_janis.xs('sigma', level = 1, axis = 1)

The reaction rate of a specific reaction is $R=\int_0^\inf dE \,\sigma\left(E\right)\phi\left(E\right)$.
On a discrete energy grid the integral simplifies to a sum: $R = \sum_E \,\sigma_E\phi_E$.
As noted above, we approximate with the time-averaged neutron spectrum : $\phi\left(E,t\right)\approx\overline{\phi_E}$.

In [7]:
def calc_reaction_rate(xs_data, energy_spectrum):
    """Calculate the reaction rate from the cross sections
    
    The reaction rate is calculated by integrating the
    product of cross-section and neutron flux over energy.
    Both are available on a discrete energy grid, which
    converts the integral into a sum.
    Reaction rates for all reactions in the cross-section
    DataFrame (xs_data) are calculted. All arrays need to 
    be on the same energy grid.
    
    The JANIS cross-section data is given in units barn.
    Therefore the one-group cross-section is multiplied with
    1e-24 to transform to SI units.
    
    Parameters
    ----------
    xs_data : pd.DataFrame
        Columns contain energy dependent cross section data in 
        units of barn.
    energy_spectrum : pd.DataFrame or pd.Series
        Columns contain the energy spectrum counts.
    
    Returns
    -------
    rate : pd.Series
        One reaction rate for each reactin in xs_data
    """
    df = pd.DataFrame()
    for name, column in xs_data.items():
        prod = energy_spectrum.values * column.values
        df[name] = prod
    rate = df.sum(axis=0)
    return rate * 1e-24

#### Isotopic Vector

The solution to the neutron transport equation (see above) is $\vec{N}(t) = \exp\left(\textbf{A}t\right)\vec{N}(0),$ approximating that the elements of $\textbf{A}$ are constant in time.

In [8]:
def isotopic_vector(matrix, t, xs, spectrum, n_0):
    """Calculate the isotopic vector evolution
    
    Uses the simplified burnup matrix to calculate
    the time evolution of the isotopic vector.
    
    Parameters
    ----------
    matrix : callable
        Simplified burnup matrix
    t : float
        Time in seconds (time the reactor is operational)
    xs : pd.DataFrame
        Cross-section data for the reactions accounted
        for in the burnup matrix
    spectrum : np.ndarray or pd.DataFrame
        Neutron spectrum, needs to be on the same energy
        grid as the cross-sections
    n_0 : np.ndarray
        Isotopic vector of the element at t=0, commonly
        the natural isotopic composition.
        
    Returns
    -------
    iso_vec : np.ndarray
        Isotopic vector at time t
    """
    reac_rate = calc_reaction_rate(xs, spectrum)
    bu_matrix = matrix(*reac_rate.values)
    exp_matrix = expm(bu_matrix * t)
    iso_vec = np.dot(exp_matrix, n_0)
    return iso_vec

#### Transition Matrices

To calulate the isotopic ratios of titanium-48/titanium-49 and boron-10/boron-11, we use the matrices:
$$
    \textbf{A}_{ti} = 
        \left(\begin{array}{ccc}
            -\sigma_{^{47}Ti\rightarrow^{48}Ti}\phi & 0 & 0 \\                                               
            \sigma_{^{47}Ti\rightarrow^{48}Ti}\phi & -\sigma_{^{48}Ti\rightarrow^{49}Ti}\phi & 0 \\ 
            0 & \sigma_{^{48}Ti\rightarrow^{49}Ti}\phi & -\sigma_{^{49}Ti\rightarrow^{50}Ti}\phi  \\ 
        \end{array}\right) 
$$
and
$$
    \textbf{A}_{b} =  
        \left(\begin{array}{cc}                                
            -\sigma_{^{10}B\rightarrow^{11}B}\phi & 0 \\ 
            0 & 0 \\ 
        \end{array}\right).
$$
Although there are other reaction modes and other titanium isotopes, we found they were negligible.

In [9]:
ti_reacs = ['Ti47_MT102', 'Ti48_MT102', 'Ti49_MT102']
b_reacs = ['B10_MT107']

In [10]:
def titanium_matrix(*args):
    """Create the transition matrix for titanium"""
    t = np.array([[-args[0],        0,        0],
                  [ args[0], -args[1],        0],
                  [       0,  args[1], -args[2]]])
    return t

def boron_matrix(*args):
    """Create the transition matrix for boron"""
    b = np.array([[-args[0], 0],
                  [    0.  , 0]])
    return b

The natural isotopic composition of boron and titanium (note that we neglected $^{46}$Ti and $^{50}$Ti) 

In [11]:
n_0_ti = np.array([0.0744,0.7372,0.0541])
n_0_b = np.array([0.2, 0.8])

## Longterm Plutonium Production

As mentioned above, our model assumes that the reactor was operated for multiple cycles (multiple batches of fuel) under approximately identical conditions.
The plutonium-per-unit-fluence is approximately constant: $\frac{Pu_0}{\Phi_0} \approx \frac{Pu(t)}{\Phi(t)}$.
Since $\Phi(t)\approx\overline{\phi}\cdot t$, longterm plutonium can be written as: $$Pu(t)\approx\frac{Pu_0\times\overline{\phi}\times t}{\Phi_0}$$

In [12]:
def plutonium_to_time(pu, flux_average, phi_0, pu_0):
    """Approximate time in units of plutonium
    
    With the assumption that plutonium-per-unit-fluence is constant for 
    an average batch of fuel (one simulation), the total plutonium 
    over several subsequent batches is related to the operating time
    of the reactor via a linear equation.
    
    Parameters
    ----------
    pu : float
        Plutonium density in g cm-3.
    flux_average : float
        Average flux in the reactor in s-1 cm-2.
    phi_0 : float
        Fluence of an average batch in cm-2.
    pu_0 : float
        Plutonium density of an average batch in g cm-3.
    
    Returns
    -------
    t : float
        Total irradiation time in s.
    """
    t = pu * phi_0 / pu_0 / flux_average
    return t

## Isotopic Ratio as a Function of Produced Plutonium

Combining the time evolution of the isotopic vector with the approximation for the longterm plutonium production allows us to define a function $\textbf{R(Pu)}$ that calculates the isotopic ratio depending on the produced plutonium.

In [13]:
def ratio_plutonium_function(spectrum, phi_0, pu_0, cross_sections,
                             matrix, n_0, idx):
    """Calculate the isotopic vector as a function of plutonium
    
    Combine steps 1 and 2 of the irm analysis. First compute the 
    isotopic vector as a function of reactor operating time, then
    insert the approximation between longterm plutonium production.
    
    Parameters
    ----------
    spectrum : np.ndarray or pd.DataFrame
        Average neutron spectrum on the same energy grid
        as the cross_sections.
    phi_0 : float
        Fluence of an average batch in cm-2.
    pu_0 : float
        Plutonium density (g cm-3) in the fuel at the end of an
        average batch. 
    cross_sections : pd.DataFrame
        Cross-sections of the reactions accounted for in the 
        burnup matrix.
    matrix : callable
        The simplified burnup matrix for the isotopic vector
        of the indicator element.
    n_0 : np.ndarray
        The natural isotopic vector of the indicator element.
    idx : list or array, len = 2
        The components of the isotopic vector that are divided to 
        calculate the ratio.
        
    Returns
    -------
    ratio : callable
    """
    flux_average = spectrum.sum()
    def ratio(pu):
        """Callable ratio function with plutonium as variable"""
        t = plutonium_to_time(pu, flux_average, phi_0, pu_0)
        iso_vec = isotopic_vector(matrix,
                                  t,
                                  cross_sections,
                                  spectrum,
                                  n_0
                                  )
        return iso_vec[idx[0]] / iso_vec[idx[1]]
    return ratio

In [14]:
ti_pu_func = ratio_plutonium_function(phi_E_mean, phi_0, pu_0,
                                      xs_values[ti_reacs], titanium_matrix,
                                      n_0_ti, [1, 2])

b_pu_func = ratio_plutonium_function(phi_E_mean, phi_0, pu_0,
                                     xs_values[b_reacs], boron_matrix,
                                      n_0_b, [0, 1])

## Infer Plutonium from Isotopic Ratio

Inverting the above function numerically allows us to estimate the produced plutonium from a (hypothetical) isotope ratio measurement.

In [15]:
def plutonium_solver(func, ratio, guess):
    """Solve equation for plutonium given an isotopic ratio
    
    Uses scipy.optimize.fsolve to solve the equation:
    
                Ratio(Pu) - Ratio_measured = 0.
    
    Parameters
    ----------
    func : callable 
        Function relating the isotopic ratio with the total plutonium
        production.
    ratio : float
        Measured isotopic ratio.
    guess : float
        Starting guess for the solver.
        
    Returns
    -------
    pu_solve
    """
    def solve_func(pu):
        return func(pu) - ratio
    pu_solve = fsolve(solve_func, guess, full_output=True)
    return pu_solve[0]

In [16]:
ti_pu_solved = plutonium_solver(ti_pu_func, 12.92, 0.023)
print(f'Ti-48/Ti-49 = 12.92 -> {ti_pu_solved[0]:.4f} g/cm3')

Ti-48/Ti-49 = 12.92 -> 0.0210 g/cm3


In [17]:
b_pu_solved = plutonium_solver(b_pu_func, 0.04, 0.023)
print(f'B-10/B-11 = 0.04 -> {b_pu_solved[0]:.4f} g/cm3')

B-10/B-11 = 0.04 -> 0.0219 g/cm3
